# Challenge 3: Standard Enhancement Multi-Agent System

## Overview
This notebook implements the multi-agent AI system for reviewing, suggesting, and validating updates to AAOIFI standards as specified in Challenge 3 of the hackathon.

### System Architecture
Our solution employs a sophisticated multi-agent architecture designed for the nuanced analysis and enhancement of Islamic financial standards. Each agent is specialized in a distinct aspect of standards processing:

We'll focus on developing three specialized agents:
1. **Review & Extraction Agent**: Processes the selected standards to extract key elements, definitions, principles, scope, recognition criteria, measurement rules, and disclosure requirements.
2. **Enhancement Agent**: Suggests modifications/enhancements by comparing with other standards, analyzing contemporary financial instruments, and identifying improvement areas.
3. **Validation Agent**: Validates proposed changes for compliance with AAOIFI principles and Shariah requirements.

### Data Flow
- **Input**: AAOIFI Financial Accounting Standards (FAS) documents
- **Processing**: Sequential processing through our three specialized agents
- **Output**: Comprehensive analysis, enhancement suggestions, and validation results

### Standards Selection
For this implementation, we'll focus on **multiple standards** to demonstrate the system's versatility:
- **FAS 4** (Murabaha and Murabaha to the Purchase Orderer)
- **FAS 10** (Istisna'a and Parallel Istisna'a)
- **FAS 32** (Ijarah and Ijarah Muntahia Bittamleek)

These standards represent diverse Islamic financial instruments, providing a robust test for our system.

## 1. Environment Setup

First, let's import the necessary libraries and initialize our environment. Our system requires:

- **Standard libraries**: For file handling, data structures, and typing
- **LangChain components**: For building our Retrieval-Augmented Generation (RAG) system
- **Google Gemini integration**: For state-of-the-art language model capabilities
- **Vector database**: For efficient document retrieval

We'll also set up important configurations like project paths and API validation.

In [1]:
# Standard imports
import os
import sys
import json
from pathlib import Path
from typing import List, Dict, Any, Optional
from dotenv import load_dotenv

# LangChain imports
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools import BaseTool
from langchain.schema import BaseMessage, AIMessage, HumanMessage

# Load environment variables (for Google API key)
load_dotenv()

# Configure paths
notebook_dir = Path(__file__).parent if "__file__" in globals() else Path.cwd()
project_root = notebook_dir.parent.parent  # Go up two levels from notebooks/callenge3 to project root
data_dir = project_root / "data"
vector_db_dir = project_root / "vector_db" / "standards_enhancement"

# Check if Google API key is set
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY environment variable is not set. Please set it to use Gemini models.")

print("Setup complete!")

e:\conda\envs\chall3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Setup complete!


## 2. PDF Processing and Vector DB Setup

A critical component of our system is the ability to process and understand complex financial standard documents. To accomplish this, we'll:

1. **Load PDF documents**: Process all relevant AAOIFI standards
2. **Extract and enrich metadata**: Identify standard types, numbers, and source information
3. **Split into semantic chunks**: Break documents into manageable pieces while preserving meaning
4. **Create vector embeddings**: Using Google's Gemini embedding model for semantic understanding
5. **Build a vector database**: For efficient similarity search and context retrieval

This approach allows our agents to quickly access the most relevant parts of the standards when performing analysis and making recommendations.

In [ ]:
def process_pdf(pdf_path):
    """Load and split a PDF into chunks.
    
    This function processes a PDF file by:
    1. Loading the document page by page
    2. Extracting metadata from the filename
    3. Adding metadata to each page
    4. Splitting into semantically meaningful chunks
    
    Args:
        pdf_path: Path to the PDF file
        
    Returns:
        List of document chunks with metadata
    """
    # Load PDF
    print(f"Loading {pdf_path}...")
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    
    # Extract metadata from filename
    filename = os.path.basename(pdf_path)
    standard_type = "FAS" if "FAS" in filename else "SS"
    standard_num = "".join(filter(str.isdigit, filename))
    
    # Add metadata to pages
    for page in pages:
        page.metadata["standard_type"] = standard_type
        page.metadata["standard_num"] = standard_num
        page.metadata["source"] = filename
        # Keep existing page number metadata
    
    # Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", ".", "!", "?", ";", ",", " ", ""]
    )
    chunks = text_splitter.split_documents(pages)
    print(f"Split {len(pages)} pages into {len(chunks)} chunks")
    
    return chunks

def create_vector_db():
    """Process PDF files and create a vector database using Gemini embeddings.
    
    This function:
    1. Checks for existing vector database to avoid reprocessing
    2. Processes multiple standards PDFs (FAS and Shariah Standards)
    3. Creates embeddings using Google's Gemini model
    4. Builds and persists a Chroma vector database
    
    Returns:
        Chroma vector database instance for document retrieval
    """
    # Create vector DB directory if it doesn't exist
    vector_db_dir.mkdir(parents=True, exist_ok=True)
    
    # Check if vector DB already exists
    if os.path.exists(vector_db_dir / "chroma.sqlite3"):
        print("Vector DB already exists. Loading existing DB...")
        # Setup Gemini embeddings
        embeddings = GoogleGenerativeAIEmbeddings(
            model="models/embedding-001"  # Gemini embedding model
        )
        # Load existing DB
        vector_db = Chroma(embedding_function=embeddings, persist_directory=str(vector_db_dir))
        return vector_db
    
    # Process standard files (focusing on FAS 10 and related SS)
    pdf_files = [
        data_dir / "FAS10.PDF",
        data_dir / "FAS 10 & SS 11.pdf",  # Related Shariah Standard
        data_dir / "SS9.pdf",  # Related Shariah Standard
        data_dir / "SS12.pdf",  # Related Shariah Standard
        data_dir / "FAS32.pdf",  # For comparison
        data_dir / "FAS4.PDF"    # For comparison
    ]
    
    # Process each PDF and collect all chunks
    all_chunks = []
    for pdf_file in pdf_files:
        if pdf_file.exists():
            chunks = process_pdf(str(pdf_file))
            all_chunks.extend(chunks)
        else:
            print(f"Warning: {pdf_file} not found")
    
    if not all_chunks:
        raise ValueError("No documents were processed. Check file paths.")
    
    # Setup Gemini embeddings
    embeddings = GoogleGenerativeAIEmbeddings(
        model="models/embedding-001"  # Gemini embedding model
    )
    
    # Create vector DB
    vector_db = Chroma.from_documents(
        documents=all_chunks,
        embedding=embeddings,
        persist_directory=str(vector_db_dir)
    )
    vector_db.persist()
    print(f"Created vector DB with {len(all_chunks)} chunks using Gemini embeddings")
    
    return vector_db

# Create or load the vector database
vector_db = create_vector_db()

Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\FAS10.PDF...
Split 42 pages into 151 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\FAS 10 & SS 11.pdf...
Split 42 pages into 151 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\FAS 10 & SS 11.pdf...
Split 23 pages into 23 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\SS9.pdf...
Split 23 pages into 23 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\SS9.pdf...
Split 34 pages into 157 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\SS12.pdf...
Split 34 pages into 157 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\SS12.pdf...
Split 70 pages into 307 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\isdb-hackthon\data\FAS32.pdf...
Split 70 pages into 307 chunks
Loading C:\Users\Zinou\OneDrive\Desktop\2CS_IASD\ISDB\

C:\Users\Zinou\AppData\Local\Temp\ipykernel_11828\3383905415.py:80: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vector_db.persist()


## 3. Define the Multi-Agent System

Our solution's core is a sophisticated multi-agent system designed for collaborative intelligence. Each agent has specialized capabilities but works as part of an integrated system.

The agents communicate through a sequential process where:
- Output from the Review Agent becomes input for the Enhancement Agent
- Output from both becomes input for the Validation Agent

### Key Components:
1. **Language Model**: Google's Gemini 1.5 Pro for advanced reasoning
2. **Vector Retrieval**: For providing relevant context to each agent
3. **Specialized Prompts**: Custom instructions for each agent's unique role
4. **Pipelines**: Structured data flow between system components

In [ ]:
# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",  # Using Gemini's most capable model
    temperature=0,          # Setting temperature to 0 for maximum determinism
    convert_system_message_to_human=True  # Required for Gemini compatibility
)

# Create a retriever from the vector DB
# This retriever uses similarity search to find the most relevant context
# from our standards database when prompted by an agent
retriever = vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}  # Retrieve more documents for comprehensive analysis
)

### 3.1 Agent 1: Review & Extraction Agent

The first agent in our system serves as the foundation for subsequent analysis. This agent has three critical responsibilities:

1. **Deep Document Analysis**: Thoroughly parsing standard documents to understand their structure and content
2. **Key Element Extraction**: Identifying and extracting specific components like definitions, principles, recognition criteria, etc.
3. **Ambiguity Detection**: Identifying areas that may be unclear or open to interpretation

The agent uses a specialized prompt template that guides it through this structured analysis process. The retrieved context provides the necessary information about the standard being analyzed.

In [ ]:
# Define the Review & Extraction Agent

# This template guides the agent to extract specific elements from a standard
# The structured approach ensures consistency across different standards
review_template = """
You are a specialized AI agent for reviewing and extracting key information from AAOIFI Financial Accounting Standards (FAS).
Your task is to comprehensively analyze the standard {standard_name} and extract its key elements.

Based on the following context information about the standard, please extract and organize the following elements:

Context information:
{context}

Please extract and organize the following:

1. Definitions of key terms in {standard_name}
2. Scope and applicability of the standard
3. Core principles and concepts
4. Recognition criteria for contracts/transactions covered by this standard
5. Measurement rules and methodologies
6. Disclosure requirements
7. Areas that are potentially unclear or ambiguous
8. Special cases or exceptions mentioned

Format your response as a detailed structured analysis that another AI agent could use to propose improvements.
"""

# Create the RAG pipeline for the Review Agent
# This is a Retrieval-Augmented Generation (RAG) chain that:
# 1. Takes a standard name as input
# 2. Retrieves relevant context from our vector database
# 3. Formats the prompt with the standard name and context
# 4. Sends the prompt to our LLM
# 5. Extracts the response as a string
review_prompt = ChatPromptTemplate.from_template(review_template)
review_chain = (
    {"context": lambda x: retriever.invoke(f"Analyze and extract key elements from {x['standard_name']}"), 
     "standard_name": lambda x: x["standard_name"]}
    | review_prompt
    | llm
    | StrOutputParser()
)

# Function to run the review agent
def run_review_agent(standard_name):
    """Run the review agent to extract information from the standard.
    
    Args:
        standard_name: Name of the standard to analyze (e.g., "FAS 10 (Istisna'a and Parallel Istisna'a)")
        
    Returns:
        Structured analysis of the standard as text
    """
    return review_chain.invoke({"standard_name": standard_name})

### 3.2 Agent 2: Enhancement Agent

The second agent in our system takes the output from the Review Agent and proposes meaningful improvements. This agent focuses on:

1. **Gap Analysis**: Identifying areas where the standard could be improved
2. **Contemporary Relevance**: Suggesting updates to align with modern financial practices
3. **Clarity Improvements**: Proposing clearer language and additional examples
4. **Global Alignment**: Identifying opportunities for harmonization with global standards while maintaining Shariah compliance

This agent combines the structured analysis from the Review Agent with additional comparative context to generate specific, actionable enhancement proposals.

In [ ]:
# Define the Enhancement Agent

# This template provides detailed instructions for suggesting improvements
# It structures the enhancement process into 8 key areas
enhancement_template = """
You are a specialized AI agent for suggesting improvements to AAOIFI Financial Accounting Standards (FAS).
Your task is to analyze the extracted information from {standard_name} and suggest meaningful enhancements.

Standard information extracted by the Review Agent:
{standard_info}

Additional context from related standards:
{context}

Please propose enhancements in the following areas:

1. Clarifications for ambiguous or unclear sections
2. Additional examples or illustrations for complex concepts
3. Updates to align with contemporary financial instruments or practices
4. Potential harmonization with global standards (like IFRS) while maintaining Shariah compliance
5. Additional guidance for practical implementation
6. Improvements to disclosure requirements
7. Addressing gaps in coverage of specific transaction types
8. Enhanced structure or organization for better usability

For each enhancement suggestion:
- Clearly identify the specific section/aspect of {standard_name} being addressed
- Explain the current limitation or issue
- Provide the proposed enhancement with specific wording where appropriate
- Justify why this enhancement would improve the standard's usability or effectiveness

IMPORTANT: Ensure all suggestions maintain Shariah compliance and align with Islamic financial principles.
"""

# Create the RAG pipeline for the Enhancement Agent
# This chain:
# 1. Takes standard info from the Review Agent and the standard name
# 2. Retrieves comparative context from related standards
# 3. Generates enhancement suggestions using all available information
enhancement_prompt = ChatPromptTemplate.from_template(enhancement_template)
enhancement_chain = (
    {"context": lambda x: retriever.invoke(f"Comparative analysis of {x['standard_name']} with other standards"), 
     "standard_info": lambda x: x["standard_info"],
     "standard_name": lambda x: x["standard_name"]}
    | enhancement_prompt
    | llm
    | StrOutputParser()
)

# Function to run the enhancement agent
def run_enhancement_agent(standard_info, standard_name):
    """Run the enhancement agent to suggest improvements to the standard.
    
    Args:
        standard_info: Structured analysis from the Review Agent
        standard_name: Name of the standard being enhanced
        
    Returns:
        Proposed enhancements as structured text
    """
    return enhancement_chain.invoke({"standard_info": standard_info, "standard_name": standard_name})

### 3.3 Agent 3: Validation Agent

The final agent in our system serves as a critical safeguard for Shariah compliance and practicality. This agent:

1. **Evaluates Compliance**: Assesses whether proposed enhancements maintain or improve Shariah compliance
2. **Checks Consistency**: Ensures alignment with AAOIFI's overall framework
3. **Assesses Practicality**: Evaluates whether suggestions can be realistically implemented
4. **Makes Recommendations**: Approves, suggests modifications, or rejects each proposal

This validation step is crucial to ensure that all enhancements meet the unique requirements of Islamic financial standards. The agent provides detailed justification for each decision, creating transparency in the validation process.

In [ ]:
# Define the Validation Agent

# This template guides the validation process with specific criteria
# It ensures all proposed enhancements meet Shariah requirements
validation_template = """
You are a specialized AI agent for validating proposed enhancements to AAOIFI Financial Accounting Standards (FAS).
Your task is to evaluate the suggested improvements to {standard_name} for compliance with Shariah principles and AAOIFI's overall framework.

Standard information extracted by the Review Agent:
{standard_info}

Proposed enhancements from the Enhancement Agent:
{enhancements}

Additional context from Shariah standards:
{context}

Please validate each proposed enhancement by assessing:

1. Shariah Compliance: Does the enhancement maintain or improve compliance with Islamic financial principles?
2. Consistency: Is it consistent with AAOIFI's overall framework and other standards?
3. Practicality: Would it be practical to implement in real-world Islamic financial institutions?
4. Clarity: Does it enhance or maintain the clarity of the standard?
5. Relevance: Is it relevant to contemporary Islamic finance practices?
6. Thoroughness: Does it address the identified issue comprehensively?

For each enhancement:
- Provide a rating (Approved, Approved with Modifications, or Rejected)
- Justify your decision with specific references to Shariah principles or AAOIFI requirements where applicable
- For 'Approved with Modifications', suggest the specific modifications needed
- For 'Rejected', explain the specific issues that make the enhancement inappropriate

Conclude with a summary of which enhancements should be adopted, modified, or rejected.
"""

# Create the RAG pipeline for the Validation Agent
# This chain:
# 1. Takes inputs from both previous agents and the standard name
# 2. Retrieves Shariah-specific context for validation
# 3. Evaluates each enhancement against Shariah principles
# 4. Produces a detailed validation report
validation_prompt = ChatPromptTemplate.from_template(validation_template)
validation_chain = (
    {"context": lambda x: retriever.invoke(f"Shariah compliance of {x['standard_name']} proposed enhancements"), 
     "standard_info": lambda x: x["standard_info"], 
     "enhancements": lambda x: x["enhancements"],
     "standard_name": lambda x: x["standard_name"]}
    | validation_prompt
    | llm
    | StrOutputParser()
)

# Function to run the validation agent
def run_validation_agent(standard_info, enhancements, standard_name):
    """Run the validation agent to evaluate proposed enhancements.
    
    Args:
        standard_info: Structured analysis from the Review Agent
        enhancements: Proposed improvements from the Enhancement Agent
        standard_name: Name of the standard being evaluated
        
    Returns:
        Validation results as structured text
    """
    return validation_chain.invoke({"standard_info": standard_info, "enhancements": enhancements, "standard_name": standard_name})

## 4. Multi-Agent Orchestration

Orchestration is critical for our multi-agent system to function effectively. This component:

1. **Coordinates Agent Workflow**: Manages the sequential process from review to enhancement to validation
2. **Handles Data Transfer**: Ensures each agent has the inputs it needs from previous agents
3. **Processes Multiple Standards**: Applies the workflow to each selected standard
4. **Consolidates Results**: Aggregates findings from all standards into a comprehensive output

The orchestration layer ensures our agents work together as an integrated system rather than isolated components. This coordinated approach allows for more sophisticated analysis than any single agent could provide.

In [ ]:
def run_multi_agent_system():
    """Run the complete multi-agent system for standard enhancement.
    
    This function orchestrates the entire multi-agent process by:
    1. Processing multiple standards sequentially
    2. Running each agent in the proper order for each standard
    3. Passing outputs between agents
    4. Storing and saving all results
    
    Returns:
        Dictionary containing all results for all standards
    """
    # Define the standards to analyze
    standards = [
        "FAS 4 (Murabaha and Murabaha to the Purchase Orderer)",
        "FAS 10 (Istisna'a and Parallel Istisna'a)",
        "FAS 32 (Ijarah and Ijarah Muntahia Bittamleek)"
    ]
    
    all_results = {}
    
    for standard in standards:
        print(f"\nStarting Multi-Agent System for {standard} Enhancement...\n")
        
        # Step 1: Run the Review & Extraction Agent
        print(f"AGENT 1 (Review & Extraction): Analyzing {standard}...")
        standard_info = run_review_agent(standard)
        print("\nReview & Extraction Complete!\n")
        
        # Step 2: Run the Enhancement Agent
        print(f"AGENT 2 (Enhancement): Suggesting improvements to {standard}...")
        enhancements = run_enhancement_agent(standard_info, standard)
        print("\nEnhancement Suggestions Complete!\n")
        
        # Step 3: Run the Validation Agent
        print(f"AGENT 3 (Validation): Evaluating proposed enhancements for {standard} for Shariah compliance...")
        validation_results = run_validation_agent(standard_info, enhancements, standard)
        print("\nValidation Complete!\n")
        
        # Store results for this standard
        # Create a standardized key format (e.g., "FAS4", "FAS10")
        standard_key = standard.split(' ')[0] + standard.split(' ')[1].strip('()')
        all_results[standard_key] = {
            "standard_name": standard,
            "standard_info": standard_info,
            "enhancements": enhancements,
            "validation_results": validation_results
        }
    
    # Save all results to file in JSON format for later analysis and visualization
    output_path = project_root / "notebooks" / "fas_enhancement_results.json"
    with open(output_path, "w") as f:
        json.dump(all_results, f, indent=2)
    
    return all_results

## 5. Run the Multi-Agent System

With our agents defined and orchestration in place, we're ready to execute the complete system. This process will:

1. **Process Multiple Standards**: Apply our multi-agent analysis to FAS 4, 10, and 32
2. **Generate Comprehensive Output**: Produce detailed analysis, enhancement suggestions, and validation for each standard
3. **Save Results**: Store all outputs in a structured JSON format for downstream applications

This execution represents the culmination of our multi-agent design, demonstrating how AI can assist in the complex task of enhancing Islamic financial standards.

In [25]:
# Run the full multi-agent system
results = run_multi_agent_system()

# Display summary of results
print("\n==== MULTI-AGENT SYSTEM RESULTS ====\n")

for standard_key, standard_results in results.items():
    print(f"\n----- {standard_results['standard_name']} -----\n")
    print("Standard Information Extracted:\n")
    print(standard_results["standard_info"][:300] + "...\n")
    
    print("Enhancements Proposed:\n")
    print(standard_results["enhancements"][:300] + "...\n")
    
    print("Validation Results:\n")
    print(standard_results["validation_results"][:300] + "...\n")

print("\nFull results saved to 'fas_enhancement_results.json'")


Starting Multi-Agent System for FAS 4 (Murabaha and Murabaha to the Purchase Orderer) Enhancement...

AGENT 1 (Review & Extraction): Analyzing FAS 4 (Murabaha and Murabaha to the Purchase Orderer)...


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Review & Extraction Complete!

AGENT 2 (Enhancement): Suggesting improvements to FAS 4 (Murabaha and Murabaha to the Purchase Orderer)...


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Enhancement Suggestions Complete!

AGENT 3 (Validation): Evaluating proposed enhancements for FAS 4 (Murabaha and Murabaha to the Purchase Orderer) for Shariah compliance...


...


Review & Extraction Complete!

AGENT 2 (Enhancement): Suggesting improvements to FAS 32 (Ijarah and Ijarah Muntahia Bittamleek)...


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Enhancement Suggestions Complete!

AGENT 3 (Validation): Evaluating proposed enhancements for FAS 32 (Ijarah and Ijarah Muntahia Bittamleek) for Shariah compliance...


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Validation Complete!


==== MULTI-AGENT SYSTEM RESULTS ====


----- FAS 4 (Murabaha and Murabaha to the Purchase Orderer) -----

Standard Information Extracted:

## Analysis of AAOIFI FAS 4 (Murabaha and Murabaha to the Purchase Orderer) based on provided context

The provided context offers limited information specifically about the *content* of FAS 4. It primarily focuses on the standard's development history and its grouping with other standards like Mush...

Enhancements Proposed:

## Proposed Enhancements to AAOIFI FAS 4 (Murabaha and Murabaha to the Purchase Orderer)

The following enhancements are proposed to improve the clarity, applicability, and effectiveness of FAS 4 while maintaining Shariah compliance:

**1. Clarifications for Ambiguous or Unclear Sections:**

* **Sec...

Validation Results:

## Validation of Proposed Enhancements to AAOIFI FAS 4

Here's an evaluation of the proposed enhancements to FAS 4, considering Shariah compliance, consistency, practicality, clarity

## 6. Interactive Query System

Beyond the core analysis, our system includes an interactive query capability that allows users to:

1. **Ask Questions**: Query specific aspects of the enhanced standards
2. **Access Insights**: Get detailed information about proposed enhancements and their validation
3. **Compare Standards**: Explore differences and similarities between different FAS

This component transforms our system from a one-time analysis tool into an ongoing knowledge resource that stakeholders can use to understand the nuances of Islamic financial standards and our proposed enhancements.

In [ ]:
# Define the template for our query system
# This template allows flexible questioning about any enhanced standard
query_template = """
You are an AI assistant specialized in AAOIFI standards, particularly the Financial Accounting Standards (FAS).
Use the following information to answer the user's question about {standard_name}:

Standard Information:
{standard_info}

Proposed Enhancements:
{enhancements}

Validation Results:
{validation_results}

Additional Context from Standards:
{context}

User Question: {question}

Provide a detailed, accurate response. If the user's question relates to a proposed enhancement,
make sure to indicate whether that enhancement was approved or not during validation.
"""

# Create a function to query a specific standard
def create_query_chain(standard_key):
    """Create a query chain for a specific standard.
    
    This function creates a specialized chain for each standard that:
    1. Loads the standard's data
    2. Retrieves relevant context based on the query
    3. Formats all information for the LLM
    4. Returns a detailed response
    
    Args:
        standard_key: The key for the standard (e.g., "FAS4")
        
    Returns:
        A runnable chain for querying the standard
    """
    standard_data = results[standard_key]
    query_prompt = ChatPromptTemplate.from_template(query_template)
    return (
        {"context": lambda x: retriever.invoke(f"Information about {standard_data['standard_name']}"), 
         "standard_info": lambda x: standard_data["standard_info"], 
         "enhancements": lambda x: standard_data["enhancements"], 
         "validation_results": lambda x: standard_data["validation_results"],
         "standard_name": lambda x: standard_data["standard_name"],
         "question": RunnablePassthrough()}
        | query_prompt
        | llm
        | StrOutputParser()
    )

# Dictionary of query chains for each standard
query_chains = {}
for standard_key in results.keys():
    query_chains[standard_key] = create_query_chain(standard_key)

def query_enhanced_standard(standard_key, question):
    """Query the enhanced standard with a specific question.
    
    Args:
        standard_key: Key for the standard to query (e.g., "FAS4")
        question: The user's question about the standard
        
    Returns:
        Detailed response to the question
    """
    if standard_key not in query_chains:
        return f"Error: Standard '{standard_key}' not found. Available standards are: {', '.join(query_chains.keys())}"
    return query_chains[standard_key].invoke(question)

In [ ]:
# Demonstrate the query system with example questions for each standard
# This shows how our system can answer specific questions about each standard
for standard_key in results.keys():
    print(f"\n\n===== QUERYING {standard_key} =====\n")
    
    # Define standard-specific questions that demonstrate system capabilities
    if standard_key == "FAS4":
        questions = [
            "What are the main improvements suggested for FAS 4?",  # Overview of enhancements
            "How does FAS 4 handle revenue recognition in Murabaha contracts?",  # Specific technical aspect
            "What challenges exist in the current standard regarding Murabaha to the Purchase Orderer?"  # Problem identification
        ]
    elif standard_key == "FAS10":
        questions = [
            "What are the main improvements suggested for FAS 10?",  # Overview of enhancements
            "How does FAS 10 handle revenue recognition in Istisna'a contracts?",  # Specific technical aspect
            "What challenges exist in the current standard regarding parallel Istisna'a?"  # Problem identification
        ]
    elif standard_key == "FAS32":
        questions = [
            "What are the main improvements suggested for FAS 32?",  # Overview of enhancements
            "How does FAS 32 handle recognition of Ijarah assets?",  # Specific technical aspect
            "What challenges exist in the current standard regarding Ijarah Muntahia Bittamleek?"  # Problem identification
        ]
    
    # Run the queries and display truncated results
    for question in questions:
        print(f"\nQuestion: {question}")
        answer = query_enhanced_standard(standard_key, question)
        print(f"\nAnswer: {answer[:500]}...\n")
        print("-" * 80)



===== QUERYING FAS4 =====


Question: What are the main improvements suggested for FAS 4?


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Answer: The provided documents do *not* contain proposed enhancements to FAS 4 (Murabaha and Murabaha to the Purchase Orderer).  The documents *do* discuss the *history* of FAS 4's development, mentioning it was grouped with standards on Musharaka and Mudaraba for initial development.  Other documents provide information on FAS 32 (Ijarah) and SS 12 (Mudaraba), but not proposed changes to FAS 4 itself.

Therefore, I cannot answer your question about the main improvements suggested for FAS 4 based on the...

--------------------------------------------------------------------------------

Question: How does FAS 4 handle revenue recognition in Murabaha contracts?


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Answer: The provided text excerpts from FAS 4 do *not* contain information about revenue recognition for Murabaha contracts. The excerpts focus on the standard's development history and mention Murabaha and Murabaha to the Purchase Orderer as initial areas of focus, but do not delve into the standard's specific content regarding revenue recognition or any other accounting treatment.  FAS 4, as titled in the provided excerpts, pertains to *Musharaka* financing, not Murabaha.  There appears to be a discre...

--------------------------------------------------------------------------------

Question: What challenges exist in the current standard regarding Murabaha to the Purchase Orderer?


...


Answer: The main improvements suggested for FAS 32 "Ijarah and Ijarah Muntahia Bittamleek" focus on enhancing clarity, aligning with contemporary practices, and improving practical application.  These proposed enhancements, and their validation status, are as follows:

**Approved Enhancements:**

* **Clearer Definitions:** Providing explicit definitions for Ijarah, IMBT, and Operating Ijarah. This was a key area of ambiguity identified in the review.  The proposed definitions were *approved* during vali...

--------------------------------------------------------------------------------

Question: How does FAS 32 handle recognition of Ijarah assets?


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Answer: The provided text from FAS 32 does *not* specify the recognition criteria for Ijarah assets.  The analysis correctly identifies this as a gap in the standard.  It *does* mention the need to identify and separate Ijarah components within a contract, suggesting that recognition likely involves this step, but the specific criteria are not defined.

However, a *proposed enhancement* to FAS 32 addresses this gap:

**5. Additional Guidance for Practical Implementation (Recognition Criteria)**

* **Enh...

--------------------------------------------------------------------------------

Question: What challenges exist in the current standard regarding Ijarah Muntahia Bittamleek?


e:\conda\envs\chall3\lib\site-packages\langchain_google_genai\chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")



Answer: The primary challenge identified in the current standard (FAS 8) regarding Ijarah Muntahia Bittamleek (IMBT), as implied by the provided context from FAS 32, is a lack of clarity and comprehensive guidance, especially in comparison to evolving industry practices and global accounting standards.  While FAS 8 does address IMBT, the decision to supersede it with FAS 32 suggests several shortcomings.  These challenges are not explicitly listed but can be inferred from the focus of the proposed enhan...

--------------------------------------------------------------------------------


## 7. Conclusion and Next Steps

### Achievements
Our multi-agent system successfully accomplishes the challenge of enhancing AAOIFI Financial Accounting Standards through AI-powered analysis. Key achievements include:

1. **Comprehensive Standard Analysis**: Detailed extraction of key components from complex financial standards
2. **Meaningful Enhancement Proposals**: Concrete suggestions that improve clarity, practicality, and contemporary relevance
3. **Shariah-Compliant Validation**: Thorough evaluation ensuring all enhancements maintain Islamic financial principles
4. **Multiple Standard Processing**: Demonstrated versatility by successfully handling three different standards
5. **Interactive Knowledge Access**: Creation of a query system to access insights about enhanced standards

### Value to Stakeholders
This system provides significant value to various stakeholders in the Islamic finance ecosystem:

- **Standards Bodies**: Accelerated standards development and enhancement process
- **Financial Institutions**: Clearer understanding and implementation of standards
- **Regulators**: More precise and comprehensive standards for oversight
- **Educators and Students**: Improved learning resources with clearer explanations

### Potential Next Steps
To further develop this system, we could consider:

1. **Extending the System**: Adding support for additional AAOIFI standards and Shariah Standards
2. **Improving the Retrieval System**: Implementing more sophisticated chunking and embedding approaches
3. **Adding Specialized Agents**: Creating agents for specific aspects like translation or examples generation
4. **Developing a Visual Interface**: Building a user-friendly interface for interacting with the system (a basic implementation is provided in the Streamlit app)
5. **Implementing Continuous Improvement**: Adding a feedback mechanism to incorporate user insights
6. **Integration with Standards Development Process**: Embedding the system into the official standards development workflow

## Required Libraries

### Development Environment
This notebook has been developed and tested with the following environment:

- **Python**: 3.9+
- **LangChain**: For agent construction, RAG pipeline, and orchestration
- **Google Generative AI**: For Gemini model access for both LLM and embeddings
- **ChromaDB**: For vector database functionality
- **PDF Processing**: For handling standard documents

### Installation Instructions
To run this notebook with Gemini models, you need to install the following libraries:

In [ ]:
# Installation commands

# Using pip
!pip install langchain langchain-core langchain-community python-dotenv
!pip install google-generativeai langchain-google-genai
!pip install chromadb
!pip install pypdf pdfplumber

# Using conda
# conda install -c conda-forge langchain python-dotenv
# conda install -c conda-forge chromadb
# conda install -c conda-forge google-generativeai 
# conda install -c conda-forge pypdf pdfplumber

# Note: You'll need to set up the GOOGLE_API_KEY in your environment variables
# or in a .env file in the project root directory:
# GOOGLE_API_KEY=your_api_key_here

# Additional required packages for the Streamlit visualization app:
# !pip install streamlit
# Run the viewer with: streamlit run fas_enhancement_viewer.py

## 8. Visualization with Streamlit

To complement our analysis system, we've created a Streamlit application that visualizes the results. This application:

1. **Presents Standard Analysis**: Shows the structured information extracted by our Review Agent
2. **Displays Enhancement Proposals**: Presents the suggestions from our Enhancement Agent
3. **Summarizes Validation Results**: Clearly indicates which enhancements were approved, modified, or rejected
4. **Provides Search Functionality**: Allows users to search across all standards and results

### Running the Visualization App

To view the results in a user-friendly interface:

1. Install Streamlit: `pip install streamlit`
2. Navigate to the notebook directory
3. Run: `streamlit run fas_enhancement_viewer.py`

This creates an interactive web application that provides a more accessible way to explore the multi-agent system's outputs.

In [ ]:
# Display a sample screenshot of the Streamlit app
from IPython.display import Image, display

try:
    # Path to a screenshot of the app (if available)
    # display(Image("fas_viewer_screenshot.png"))
    # If no screenshot is available:
    print("Visualization App Features:")
    print("- Tab-based interface for standard information, enhancements, and validation results")
    print("- Color-coded display of approved, modified, and rejected enhancements")
    print("- Search functionality across all standards")
    print("- Sidebar navigation between different standards")
    print("- Expandable sections for detailed information")
    print("\nRun 'streamlit run fas_enhancement_viewer.py' to launch the interactive app")
except:
    print("Visualization app screenshot not available. Run the app using streamlit to see the interface.")